In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/input-outliers'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**LOADING PREPROCESSED DATA IN DASK DATAFRAME**

In [ ]:
import dask.dataframe as dd
df_test = dd.read_csv("/kaggle/input/data11/fdata_test.csv")
df_train = dd.read_csv("/kaggle/input/data11/fdata_train.csv")

In [ ]:
Y_train = df_train['HasDetections']
X_train = df_train.iloc[:,:-1]
X_tester = df_test.iloc[:,:-1]

**CONVERTING TO DASK ARRAYS**

In [ ]:
X_train_d = X_train.to_dask_array(lengths=True)
Y_train_d= Y_train.to_dask_array(lengths=True)
X_tester_d = X_tester.to_dask_array(lengths=True)


**CONVERTING TO DMATRIX**

In [ ]:
import xgboost as xgb

dtrain2 = xgb.DMatrix(X_train_d, label=Y_train_d)
dtest2 = xgb.DMatrix(X_tester_d)

In [ ]:
#conda install -c conda-forge dask-ml
#X_test_d = X_test.to_dask_array(lengths=True)
#Y_test_d= Y_test.to_dask_array(lengths=True)
#import xgboost as 
#from sklearn.calibration import CalibratedClassifierCV
#clf = xgb.XGBClassifier()
#mp=CalibratedClassifierCV(clf, method='isotonic', cv=2)
#import xgboost as xgb
#clf = xgb.XGBClassifier(n_estimators =100,max_depth=6)
#clf.fit(X_train_d, Y_train_d)
#testPredictions = metLearn.predict(test)
#Y_pred=clf.predict(X_test_d)
#Y_pred2=clf.predict(X_tester_d)
#dtrain = xgb.DMatrix(X_train_d, label=Y_train_d)
#dtest = xgb.DMatrix(X_test_d, label=Y_test_d)
#dtest2 = xgb.DMatrix(X_tester_d)
#roc_auc_score(Y_test_d,Y_pred)
#Y_pred2 =model
#_pred

**LOADING THE MACHINE IDENTIFIER COLUMN**

In [ ]:
from sklearn.metrics import roc_auc_score
d_test =  dd.read_csv("/kaggle/input/save-the-attack-contest/test_data.csv")
d_new = d_test['MachineIdentifier'].compute()

**PARAMETERS DICTIONARY FOR XGBOOST**

In [ ]:
params = {
    # Parameters that we are going to tune.
    'max_depth':4,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 0.5,
    'colsample_bytree': 1,
    # Other parameters
    'num_parallel_tree':3,
    'objective':'binary:logistic',
}
params['eval_metric']= "auc"
num_boost_round = 400

**XGB MODEL TRAINING**

In [ ]:
model = xgb.train(
    params,
    dtrain2,
    num_boost_round=num_boost_round,
)

**MODEL PREDICTION**

In [ ]:
ypred2=model.predict(dtest2)

**SAVING THE PREDICTION IN CSV FILE**

In [ ]:

import pandas as pd
submission = pd.DataFrame({
        "MachineIdentifier": d_new,
        "HasDetections": ypred2
            }) 

submission.to_csv('submission.csv', index=False)

**DISPLAYING AND DOWNLOADING SUBMISSION FILE**

In [ ]:
from IPython.display import FileLink
FileLink(r'submission.csv')

In [ ]:
while True:
    x = 2